In [1]:
from pyscf import gto, scf, dft, fci, mcscf, ao2mo, tools
import numpy as np
import scipy as sp
from opt_einsum import contract
np.set_printoptions(precision=8, suppress=True, linewidth=200)

In [2]:
# Fock matrix and energy for a given density in AO basis
# for doubly occupied orbitals
def fock(D, g, h):
    # print('One-electron energy: ', np.einsum('ij,ij->', h, D))
    J = contract('ijkl,kl->ij', g, D)        # Coulomb
    # print('Hartree energy: ',  .5*np.einsum('ij,ij->', J, D))
    K = contract('ilkj,kl->ij', g, D)   # Exchange
    # print('Exchange energy: ', .5*np.einsum('ij,ij->', K, D))
    if h is not None:
        F = h + J - 0.5*K             # Fock matrix
        E = .5*contract('ij,ij->', h + F, D)
    else:
        F = J - 0.5*K
        E = 0.0

    return F, E

# naive implementation of the effective Fock matrix, which corresponds to the gradient fo the energy
# I need to pass the full set of orbitals. This only works for mo quantities!
def fock_eff(h, g, D1, D2):
    Fpq  = contract('pr,rq->pq', D1, h)
    Fpq += contract('prst,qrst->pq', D2, g)
    return Fpq

# compute AO to MO transformation of two-electron integrals
# for the given set of MO coefficients C
def eri_ao2mo(C, g):
    pqrw = contract("pqrs,sw->pqrw", g   , C)
    pqvw = contract("pqrw,rv->pqvw", pqrw, C)
    puvw = contract("pqvw,qu->puvw", pqvw, C)
    tuvw = contract("puvw,pt->tuvw", puvw, C)
    return puvw, tuvw

def update_total_density(C_act, D_ao_inactive, D_mo_active):
    D_ao_active = contract("pt, qu, tu->pq", C_act, C_act, D_mo_active)
    D_ao_total = D_ao_active + D_ao_inactive
    return D_ao_total


In [29]:
mol = gto.M(
    atom = 'C  0.0      0.0      0.66242; \
            C  0.0      0.0     -0.66242; \
            H -0.12018  0.91284  1.23164; \
            H  0.12018 -0.91284  1.23164; \
            H  0.12018  0.91284 -1.23164; \
            H -0.12018 -0.91284 -1.23164;',
    basis="sto-3g",
    verbose=3
)

mol = gto.M(
    atom = 'N 0 0 0; N 0 0 2.4',
    basis = 'cc-pvdz',
    verbose=4
)

mf = scf.RHF(mol)
umf = scf.UHF(mol)

omega = 0.5
mf_dft = dft.RKS(mol)
mf_dft._numint.libxc = dft.xcfun
mf_dft.xc = f'ldaerf + lr_hf({omega})'
mf_dft.omega = omega

System: uname_result(system='Linux', node='pauli', release='6.16.1-arch1-1', version='#1 SMP PREEMPT_DYNAMIC Fri, 15 Aug 2025 16:04:43 +0000', machine='x86_64')  Threads 16
Python 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:09:02) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.0  h5py 3.14.0
Date: Mon Aug 18 17:25:43 2025
PySCF version 2.10.0
PySCF path  /home/stefano/Miniconda/lib/python3.13/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 14
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 N      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 N      0.000000000000   0.000000000000   2.400000000000 AA    0.0000000000

In [30]:
mf.kernel()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpmw7p0u7d
max_memory 4000 MB (current use 259 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -107.587189283106
  HOMO = -0.18258106053183  LUMO = -0.138974041095379
cycle= 1 E= -108.134827806919  delta_E= -0.548  |g|= 0.264  |ddm|= 1.59
  HOMO = -0.398628945522392  LUMO = -0.158745411134545
cycle= 2 E= -108.152054629078  delta_E= -0.0172  |g|= 0.0654  |ddm|= 0.32
  HOMO = -0.362261886433588  LUMO = -0.119728284239383
cycle= 3 E= -108.153406874653  delta_E= -0.00135  |g|= 0.0138  |ddm|= 0.0619
  HOMO = -0.364220418062815  LUMO = -0.121346128666775
cycle= 4 E= -108.153509434608  delta_E= -0.000103  |g|= 0.

np.float64(-108.15351020213947)

In [31]:
mf_dft.kernel()



******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpqxlyo2_n
max_memory 4000 MB (current use 260 MB)
XC library pyscf.dft.xcfun version 2.1.1
    XCFun DFT library Copyright 2009-2020 Ulf Ekstrom and contributors.
See http://dftlibs.org/xcfun/ for more information.

This is free software; see the source code for copying conditions.
There is ABSOLUTELY NO WARRANTY; not even for MERCHANTABILITY or
FITNESS FOR A PARTICULAR PURPOSE. For details see the documentation.
Scientific users of this library should cite
U. Ekstrom, L. Visscher, R. Bast, A. J. Thorvaldsen and K. Ruud;
J.Chem.Theor.Comp. 2010, DOI: 10.1021/ct100117s

XC functionals = ldaerf + lr_hf(0.5)
radial grids: 
Treutler-Ahlrich

np.float64(-107.99720983713331)

In [ ]:
umf.kernel()

In [ ]:
S = mf.get_ovlp()
C = mf.mo_coeff

D1_ao = mf.make_rdm1()

SC = S@C
D1_mo = C.T@S.T@D1_ao@S@C
D1_mo_ = contract('λp,λδ,δq->pq', SC, D1_ao, SC)

print(np.allclose(D1_mo, D1_mo_))

In [ ]:
D2_ao = mf.make_rdm2()
D2_mo = contract('λp,δq,λδστ,σr,τs->pqrs', SC, SC, D2_ao, SC, SC)


In [ ]:
h_ao = mf.get_hcore()
h_mo = C.T@h_ao@C

# both of these are rank-4 tensors
g_ao = mf.mol.intor('int2e')
g_mo = mf.mol.ao2mo(C, compact=False).reshape(mol.nao, mol.nao, mol.nao, mol.nao)


In [ ]:
# built fictitous density matrices
# _D1_mo = 2*np.eye(mol.nao)
_D1_mo = D1_mo
_D2_mo = contract('pq,rs->pqrs', _D1_mo, _D1_mo) - 0.5*contract('ps,rq->pqrs', _D1_mo, _D1_mo)

print(np.allclose(D2_mo, _D2_mo))

In [ ]:
# the off-diagonal should be zero for a converged calculation
F_eff_mo = fock_eff(h_mo, g_mo, D1_mo, D2_mo)
print(F_eff_mo)

# this is the gradient, should be zero for a converged HF calculation!
print(F_eff_mo - F_eff_mo.T)

In [ ]:
n_occ = np.count_nonzero(mf.mo_occ)
C_occ = C[:, mf.mo_occ == 2]
print(n_occ)

# let's see if the Fock effective is actually the same as the Fock computed from the mf object
F_ao = mf.get_fock()
F_mo = C.T@F_ao@C
print(2.0*F_mo)
print(F_eff_mo)

In [ ]:
umf.mol

In [ ]:
nmo_ina=7
mos_ina=[0,1,2,3,4,5,6]
nmo_act=2
mos_act=[7,8]
mos_vir=[9,10,11,12,13]

C = np.zeros_like(mf.mo_coeff)
C[:, :nmo_ina] = mf.mo_coeff[:, mos_ina]
C[:, nmo_ina:nmo_ina+nmo_act] = mf.mo_coeff[:, mos_act]
C[:, nmo_ina+nmo_act:] = mf.mo_coeff[:, mos_vir]

C_ina = C[:, :nmo_ina]
C_act = C[:, nmo_ina:nmo_ina+nmo_act]
print(C_ina)
print(C_act)

mo_occ = np.zeros_like(mf.mo_occ)
mo_occ[:nmo_ina] = mf.mo_occ[mos_ina]
mo_occ[nmo_ina:nmo_ina+nmo_act] = mf.mo_occ[mos_act]
mo_occ[nmo_ina+nmo_act:] = mf.mo_occ[mos_vir]
print(mo_occ)

In [ ]:
# getting full 1RDM in one shot
# SC = S@C
# D1_ao = mf.make_rdm1()
# D1_mo = contract('λp,λδ,δq->pq', SC, D1_ao, SC)

# getting full 1RDM by parts
D1_ina_ao = mf.make_rdm1(C_ina, mo_occ[:nmo_ina])
D1_ina_mo = contract('λp,λδ,δq->pq', S@C_ina, D1_ina_ao, S@C_ina)
print(D1_ina_mo)

D1_act_ao = mf.make_rdm1(C_act, mo_occ[nmo_ina:nmo_ina+nmo_act])
D1_act_mo = contract('λp,λδ,δq->pq', S@C_act, D1_act_ao, S@C_act)
print(D1_act_mo)

D1_tot_ao = D1_ina_ao + D1_act_ao
D1_tot_mo = contract('λp,λδ,δq->pq', SC, D1_tot_ao, SC)

print(np.allclose(D1_ao, D1_tot_ao))
print(np.allclose(D1_mo, D1_tot_mo))

In [ ]:
# getting full 2RDM in one shot
# D2_ao = mf.make_rdm2()
# D2_mo = contract('λp,δq,λδστ,σr,τs->pqrs', SC, SC, D2_ao, SC, SC)

# I cannot get the 2RDM by computing the inactive and active parts in ao, and then sum them up,
# but I can break the full 2RDM in mo basis into inactive and active, and transform those to ao basis
ina_slc = slice(0, nmo_ina)
act_slc = slice(nmo_ina, nmo_ina+nmo_act)

# D2_ina_mo = np.zeros_like(D2_mo)
# D2_ina_mo[ina_slc, ina_slc, ina_slc, ina_slc] = D2_mo[ina_slc, ina_slc, ina_slc, ina_slc]
# D2_ina_ao = contract('λt,δu,tuvw,σv,τw->λδστ', C, C, D2_ina_mo, C, C)
D2_ina_mo = D2_mo[ina_slc, ina_slc, ina_slc, ina_slc]
print(D2_ina_mo.shape)
# this does not work
# D2_ina_ao = contract('λt,δu,tuvw,σv,τw->λδστ', C_ina, C_ina, D2_ina_mo, C_ina, C_ina)

# D2_act_mo = np.zeros_like(D2_mo)
# D2_act_mo[act_slc, act_slc, act_slc, act_slc] = D2_mo[act_slc, act_slc, act_slc, act_slc]
# D2_act_ao = contract('λt,δu,tuvw,σv,τw->λδστ', C, C, D2_act_mo, C, C)
D2_act_mo = D2_mo[act_slc, act_slc, act_slc, act_slc]
print(D2_act_mo.shape)
# this does not work
# D2_act_ao = contract('λt,δu,tuvw,σv,τw->λδστ', C_act, C_act, D2_act_mo, C_act, C_act)



In [ ]:
# reproducing Delcey
nIn = nmo_ina
nAct = nmo_act

# Inactive Fock matrix
Din = np.einsum('ik,jk->ij', C[:, :nIn], C[:, :nIn]) #inactive density
Jin = np.einsum('ijkl,kl->ij', g_ao, Din)
Kin = np.einsum('ilkj,kl->ij', g_ao, Din)
Fin = h_ao + 2*Jin - Kin

# Active Fock matrix
Dact_pq = np.einsum('tu,pt,qu->pq', D1_act_mo, C_act, C_act) #Transform to AO basis
Jact = np.einsum('ijkl,kl->ij', g_ao, Dact_pq)
Kact = np.einsum('ilkj,kl->ij', g_ao, Dact_pq)
Fact = 2 * Jact - Kact

# get active quantities
puvw, _ = eri_ao2mo(C_act, g_ao)

# Form effective Fock matrix
Feff = np.zeros_like(Din)
Feff[:nIn,:] = np.einsum("pt,pq,qn->tn", C[:, :nIn], 2*Fin+Fact, C) #Fiq
Feff[nIn:nIn+nAct,:] = np.einsum("tu,pu,pq,qn->tn", D1_act_mo, C_act, Fin, C) #Ftq (1)
Feff[nIn:nIn+nAct,:] += np.einsum("quvw,tuvw,qn->tn", puvw, D2_act_mo, C) #Ftq (2)
# print(Feff)

# Extract some diagonals
diag1 = np.einsum("pq,pm,qm->m", 2*Fin+Fact, C, C) # Diagonal of 2*Fin+Fact in MO basis
diag2 = np.diagonal(Feff) # Diagonal of the effective Fock matrix

# Form Hessian diagonal
Hess = np.zeros_like(Din)
Hess[:nIn,nIn:] = 2* diag1[nIn:] - 2* diag1[:nIn].reshape(-1,1) #Sum of a line and column vectors
Hess[nIn:nIn+nAct,:] = - 2 * diag2[nIn:nIn+nAct].reshape(-1, 1)
Hess[nIn:nIn+nAct,nIn+nAct:] += np.einsum('tt,a->ta',D1_act_mo,diag1[nIn+nAct:])
Hess[nIn:nIn+nAct,:nIn] += np.einsum('tt,a->ta',D1_act_mo,diag1[:nIn])
Hess += np.transpose(Hess)
Hess[:nIn,:nIn] = 1 # To avoid division by 0
Hess[nIn + nAct:, nIn + nAct:] = 1 # To avoid division by 0
print(Hess)

In [ ]:
# build effective Fock matrix according to Delcey

# AO to MO ERI transformation
g_λuvw, g_tuvw = eri_ao2mo(C_act, g_ao)
# print(g_λuvw.shape)

_, g_mo = eri_ao2mo(C, g_ao)

# inactive density matrix in ao basis
D1_ina_ao = mf.make_rdm1(C_ina, mo_occ[:nmo_ina])

# compute the inactive Fock
F_ina_ao, E_ina = fock(D1_ina_ao, g_ao, h_ao)
print(np.allclose(F_ina_ao, Fin))
# F_ina_mo = contract('λt,λδ,δu->tu', C, F_ina_ao, C)

# compute active Fock matrix in AO basis
F_act_ao, _ = fock(D1_act_ao, g_ao, None)
print(np.allclose(2*F_act_ao, Fact))
# print(2*F_act_ao)

# effective Fock matrix
F_eff_mo = np.zeros_like(F_act_ao)
F_eff_mo[:nmo_ina,:] = contract('λi,λδ,δq->iq', C_ina, 2*F_ina_ao + 2*F_act_ao, C) # Fiq
F_eff_mo[nmo_ina:nmo_ina+nmo_act,:] = contract('tu,λu,λσ,σq->tq', D1_act_mo, C_act, F_ina_ao, C) # Ftq (1)
F_eff_mo[nmo_ina:nmo_ina+nmo_act,:] += contract('tuvw,λuvw,λq->tq', D2_act_mo, g_λuvw, C) # Ftq (2)
# print(F_eff_mo)

# Approximate Hessian
F_tot_pp = np.einsum("pq,pm,qm->m", 2*F_ina_ao + 2*F_act_ao, C, C) # Diagonal of 2*Fin+2*Fact in MO basis
print(F_tot_pp)
F_eff_pp = np.diagonal(F_eff_mo) # Diagonal of the effective Fock matrix
print(F_eff_pp)

H_app = np.zeros_like(Din)

# Hia = 2*Ftot_aa - 2*Ftot_ii
H_app[:nmo_ina, nmo_ina+nmo_act:] = 2*F_tot_pp[nmo_ina+nmo_act:] - 2*F_tot_pp[:nmo_ina].reshape(-1,1)
# Hta = Dact_tt*Fin_aa - 2*Feff_tt + Dact_tt*Fact_aa
#     = Dact_tt*(Fin_aa + Fact_aa) - 2*Feff_tt
#     = Dact_tt*(Ftot_aa) - 2*Feff_tt
H_app[nmo_ina:nmo_ina+nmo_act, nmo_ina+nmo_act:] += contract('tt,a->ta', D1_act_mo, F_tot_pp[nmo_ina+nmo_act:])
H_app[nmo_ina:nmo_ina+nmo_act, nmo_ina+nmo_act:] += -2*F_eff_pp[nmo_ina:nmo_ina+nmo_act].reshape(-1, 1)
# H_app[nmo_ina:nmo_ina+nmo_act, :] += -2*F_eff_pp[nmo_ina:nmo_ina+nmo_act].reshape(-1, 1)
# Hit = 2*Ftot_tt - 2*Ftot_ii + Dact_tt*Fin_ii - 2*Ftot_tt
H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] += 2*F_tot_pp[nmo_ina:nmo_ina+nmo_act]
H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] -= 2*F_tot_pp[:nmo_ina].reshape(-1,1)
H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] += contract('tt,i->it', D1_act_mo, F_tot_pp[:nmo_ina]) #.reshape(-1,1)
H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] -= 2*F_eff_pp[nmo_ina:nmo_ina+nmo_act]
H_app += H_app.T
H_app[:nmo_ina,:nmo_ina] = 1 # To avoid division by 0
H_app[nmo_ina:nmo_ina+nmo_act,nmo_ina:nmo_ina+nmo_act] = 1 # To avoid division by 0
H_app[nmo_ina+nmo_act:,nmo_ina+nmo_act] = 1 # To avoid division by 0
print(H_app)


In [ ]:
_F_eff_mo = fock_eff(h_mo, g_mo, D1_mo, D2_mo)
print('_F_eff_mo:\n', _F_eff_mo)


In [22]:
# active_indices is either a single list of active orbitals or a tuple with two lists
# the active orbital indices are 1-based
def casscf(mf, nel_act, nmo_act, active_mos, orb_opt=False, max_iter=100, threshold=1e-6, alpha=None, fci_solver=None, debug=False):

    # determine if the calculation is restricted or unrestricted
    if type(mf) == scf.hf.RHF or type(mf) == dft.rks.RKS:
        nspins = 1
    elif type(mf) == scf.uhf.UHF or type(mf) == dft.uks.UKS:
        nspins = 2
    else:
        raise TypeError('unsupported mf object')

    # we only support an even number of inactive electrons
    nel_ina = mf.mol.nelectron - nel_act
    assert(nel_ina % 2 == 0)
    nmo_ina = nel_ina // 2
    nmo_vir = mf.mol.nao - nmo_ina - nmo_act
    spin = mf.mol.spin
    assert(nel_act % 2 == spin % 2)
    mult = mf.mol.multiplicity
    nel_fci = ((nel_act + mult - 1)//2, (nel_act - mult + 1)//2)

    # define lists of inactive, active and virtual orbitals
    # restricted case
    if nspins == 1:
        # make sure the list of indices is the same as the number of active mos
        assert(len(active_mos) == nmo_act)
        mos_act = [i-1 for i in active_mos]

        # figure how many orbitals were occupied in the mf object
        occupied_mos = np.where(mf.mo_occ > 0.0)[0]
        # the inactive orbitals are all those that are occupied and not in active
        mos_ina = [i for i in occupied_mos if i not in mos_act]

        # virtual orbitals
        unoccupied_mos = np.where(mf.mo_occ == 0.0)[0]
        mos_vir = [i for i in unoccupied_mos if i not in mos_act]
    # unrestricted case
    elif nspins == 2:
        # figure out whether a two lists of active mo indices were passed (α and β)
        if type(active_mos) == tuple:
            assert(len(active_mos) == 2)
            assert(len(active_mos[0]) == nmo_act)
            assert(len(active_mos[1]) == nmo_act)
            mos_act = ([i-1 for i in active_mos[0]],[i-1 for i in active_mos[1]])
        # only one list was passed
        elif type(active_mos) == list:
            assert(len(active_mos) == nmo_act)
            mos_act = ([i-1 for i in active_mos],[i-1 for i in active_mos])
        else:
            raise TypeError('active_mos must be a list of length nmo_act or tuple containing two lists')

        # figure out inactive and virtual orbitals
        mos_ina = []
        mos_vir = []
        for ispin in range(nspins):
            occupied_mos = np.where(mf.mo_occ[ispin] > 0.0)[0]
            unoccupied_mos = np.where(mf.mo_occ[ispin] == 0.0)[0]

            mos_ina.append([i for i in occupied_mos if i not in mos_act[ispin]])
            mos_vir.append([i for i in unoccupied_mos if i not in mos_act[ispin]])

        # and make them a tuple
        mos_ina = tuple(mos_ina)
        assert(len(mos_ina[0]) == len(mos_ina[1]))
        mos_vir = tuple(mos_vir)


    # print info on inactive and active orbital spaces
    if nspins == 1:
        print('Inactive orbitals:', np.array(mos_ina)+1)
        print('Active orbitals:', np.array(mos_act)+1)
    elif nspins == 2:
        print('Inactive orbitals alpha:', np.array(mos_ina[0])+1)
        print('Inactive orbitals beta: ', np.array(mos_ina[1])+1)
        print('Active orbitals alpha:', np.array(mos_act[0])+1)
        print('Active orbitals beta: ', np.array(mos_act[1])+1)


    # get inactive and active MO coefficients rearranged such that
    # first come all inactive, then all active, then all virtual
    if nspins == 1:
        C = np.zeros_like(mf.mo_coeff)
        C[:, :nmo_ina] = mf.mo_coeff[:, mos_ina]
        C[:, nmo_ina:nmo_ina+nmo_act] = mf.mo_coeff[:, mos_act]
        C[:, nmo_ina+nmo_act:] = mf.mo_coeff[:, mos_vir]
    # elif nspins == 2:
        # C = np.zeros_like(mf.mo_coeff)
        # for ispin in range(nspins):
        #     C[:, :nmo_ina] = mf.mo_coeff[:, mos_ina]
    #     C_A = np.zeros((nspins, mf.mol.nao, nmo_act))
    #     C_A[0] = C[0,:,active[0]].transpose()
    #     C_A[1] = C[1,:,active[1]].transpose()

    # occupations should be rearranged as well
    mo_occ = np.zeros_like(mf.mo_occ)
    mo_occ[:nmo_ina] = mf.mo_occ[mos_ina]
    mo_occ[nmo_ina:nmo_ina+nmo_act] = mf.mo_occ[mos_act]
    mo_occ[nmo_ina+nmo_act:] = mf.mo_occ[mos_vir]
    print('Inactive occupations:\n', mo_occ[:nmo_ina])
    print('Active occupations:\n', mo_occ[nmo_ina:nmo_ina+nmo_act])

    # total density matrices in AO basis made from the HF density
    S = mf.get_ovlp()
    C_act = C[:, nmo_ina:nmo_ina+nmo_act]
    D1_act_ao = mf.make_rdm1(C_act, mo_occ[nmo_ina:nmo_ina+nmo_act])
    D1_act_mo = contract('λp,λδ,δq->pq', S@C_act, D1_act_ao, S@C_act)
    # D2_ao = mf.make_rdm2()
    # transform them to MO basis
    # SC = S@C
    # these are full size!
    # D1_mo = contract('δq,δσ,σr->qr', SC, D1_ao, SC)
    # D2_mo = contract('λp,δq,λδστ,σr,τs->pqrs', SC, SC, D2_ao, SC, SC)

    # in the initial step, the active density matrix is equal to the
    # HF density matrix within the active subspace
    # if nspins == 1:
    #     D_A_mo = np.zeros((nmo_act,nmo_act))
    #     for i in range(np.count_nonzero(mo_occ_active)):
    #         D_A_mo[i,i] = 2.0
    # else:
    #     D_A_mo = np.zeros((nspins,nmo_act,nmo_act))
    #     for ispin in range(nspins):
    #         for i in range(np.count_nonzero(mo_occ_active[ispin])):
    #             D_A_mo[ispin,i,i] = 1.0

    # get kinetic energy integrals
    h = mf.get_hcore()

    # check if it is a range-separated calculation
    if hasattr(mf, 'omega'):
        omega = mf.omega
    else:
        omega = None

    # get lr two-body MO integrals
    # neri = nmo_act*(nmo_act+1)//2
    with mf.mol.with_range_coulomb(omega=omega):
        if nspins == 1:
            g = mol.intor('int2e')
        # else:
            # g = np.zeros((3, neri, neri))
            # g[0] = mf.mol.ao2mo(C_A[0])
            # g[1] = mf.mol.ao2mo((C_A[0], C_A[0], C_A[1], C_A[1]))
            # g[2] = mf.mol.ao2mo(C_A[1])

    # get nuclear repulsion energy
    E_nuc = mf.mol.get_enuc()

    # create the FCI solver
    if fci_solver is None:
        if nspins == 1:
            fci_solver = fci.direct_spin1.FCI()
        else:
            fci_solver = fci.direct_uhf.FCI()
    # fci_solver = fci.addons.fix_spin_(fci_solver, ss=spin)

    # initialize history lists
    D1_act_mo_history = []
    E_history = []

    n_iter = 0
    D1_act_mo_history.append(D1_act_mo)
    E_history.append(mf.e_tot)
    converged = False
    # start the iterative loop
    print(f'\nIter    <Ψ|S^2|Ψ>    Corr. energy      Total energy          |G|')
    while n_iter < max_iter:
        n_iter += 1

        # get active MOs
        C_ina = C[:, :nmo_ina]
        C_act = C[:, nmo_ina:nmo_ina+nmo_act]

        # AO to MO ERI transformation
        g_λuvw, g_tuvw = eri_ao2mo(C_act, g)

        # _, g_mo = eri_ao2mo(C, g)

        # inactive density matrix in ao basis
        D1_ina_ao = mf.make_rdm1(C_ina, mo_occ[:nmo_ina])

        # compute the inactive Fock
        F_ina_ao, E_ina = fock(D1_ina_ao, g, h)
        F_ina_mo = contract('λt,λδ,δu->tu', C_act, F_ina_ao, C_act)

        # update total density matrix
        D1_tot_ao = D1_ina_ao + D1_act_ao

        # get the KS potential and LR active components to subtract
        F_ks = mf.get_fock(dm=D1_tot_ao)   # this is ks_mat
        J_act_lr, K_act_lr = mf.get_jk(dm=D1_act_ao, omega=omega)
        if nspins == 1:
            V_emb = F_ks - (J_act_lr - 0.5*K_act_lr)  # J_A_lr - 0.5*K_A_lr is ks_ref
        # else:
        #     # sum alpha and beta Coulopmb contributions to each other
        #     J_A_lr[:] += J_A_lr[::-1]
        #     V_emb = (F_ks - (J_A_lr - K_A_lr))
        # notice that the above calls to the mf objects take into account both the
        # CI contributions and the MOs update, because those are hidden in the density matrix


        # transform embedding potential to active MO basis
        if nspins == 1:
            V_emb_mo = contract('pt,pq,qu->tu', C_act, V_emb, C_act)
        # else:
        #     V_emb_mo = np.zeros((2, nmo_act, nmo_act))
        #     for ispin in range(nspins):
        #         V_emb_mo[ispin] = np.einsum('pt,pq,qu->tu', C_A[ispin], V_emb[ispin], C_A[ispin])
        #         if debug:
        #             print(f'V_emb_mo[{ispin}] =\n', V_emb_mo[ispin])

        # compute inactive energy
        E_ina = mf.energy_tot(dm=D1_tot_ao) - E_nuc
        if nspins == 1:
            E_ina -=    contract('pq,pq->', D1_act_ao, F_ks)
            E_ina += .5*contract('pq,pq->', D1_act_ao, (J_act_lr - 0.5*K_act_lr))
        # else:
        #     for ispin in range(nspins):
        #         E_I -=    np.einsum('pq,pq->', D_A[ispin], F_ks[ispin])
        #         E_I += .5*np.einsum('pq,pq->', D_A[ispin], (J_A_lr[ispin] - K_A_lr[ispin]))

        # solve the FCI problem (with the embedding potential and lr ERI)
        E_act, CI_vec = fci_solver.kernel(h1e=V_emb_mo, eri=g_tuvw, norb=nmo_act, nelec=nel_fci)
        # for CASSCF, we just use F_ina_mo instead of the embedding potentials
        # E_act, CI_vec = fci_solver.kernel(h1e=F_ina_mo, eri=g_tuvw, norb=nmo_act, nelec=nel_fci)
        if debug:
            print('\nInactive energy:', E_ina + E_nuc)
            print('Active energy:', E_act)

        # new active density matrix
        if nspins == 1:
            D1_act_mo, D2_act_mo = fci_solver.make_rdm12(CI_vec, norb=nmo_act, nelec=nel_fci)

            # density damping
            # if alpha is not None:
            #     D_A_mo = (1-alpha)*D_A_mo + alpha*D_A_history[-1]

            # transform D1_act_mo to AO basis
            D1_act_ao = contract('λt,tu,δu->λδ', C_act, D1_act_mo, C_act)

            SS, Ms = fci.spin_square(CI_vec, norb=nmo_act, nelec=nel_fci)
        # else:
        #     # D_A_mo = fci_solver.make_rdm1s(CI_vec, norb=nmo_act, nelec=nel_fci)
        #     D_A_mo, d2_A_mo = fci_solver.make_rdm12s(CI_vec, norb=nmo_act, nelec=nel_fci)
        #     if alpha is not None:
        #         D_A_mo_a = (1-alpha)*D_A_mo[0] + alpha*D_A_mo_history[-1][0]
        #         D_A_mo_b = (1-alpha)*D_A_mo[1] + alpha*D_A_mo_history[-1][1]
        #         D_A_mo = (D_A_mo_a, D_A_mo_b)
        #     # transform D_AO_mo to AO basis
        #     # for ispin in range(nspins):
        #         # D_A[ispin] = np.einsum('pt,tu,qu->pq', C_A[ispin], D_A_mo[ispin], C_A[ispin])

        #     SS, Ms = fci.spin_op.spin_square_general(*D_A_mo, *d2_A_mo, C_A)

        # update energy and D1 history
        E_history.append(E_ina + E_act + E_nuc)
        error = E_history[-1] - E_history[-2]
        E_corr = E_history[-1] - E_history[0]
        D1_act_mo_history.append(D1_act_mo)
        if debug:
            if nspins == 1:
                print('\nActive density matrix:\n', D1_act_mo)
            # else:
            #     print('\nAlpha active density matrix:\n', D_A_mo[0])
            #     print('\nBeta active density matrix:\n', D_A_mo[1])
            #     occ_a,mos_a = np.linalg.eigh(D_A_mo[0])
            #     occ_b,mos_b = np.linalg.eigh(D_A_mo[1])
            #     print('\nAlpha active orbitals:\n', mos_a[:,0])
            #     print('\nAlpha active orbitals:\n', mos_a)
            #     print('\nBeta active orbitals:\n', mos_b)
            #     print('Occupation numbers:\n', np.asarray([occ_a, occ_b]), '\n')
            #     print('Total number of el:\n', np.sum(np.asarray([occ_a, occ_b]),axis=1), '\n')

        if orb_opt:
            # compute active Fock matrix in AO basis, needed to compute the orbital gradient
            F_act_ao, _ = fock(D1_act_ao, g, None)

            # effective Fock matrix
            F_eff_mo = np.zeros_like(F_act_ao)
            F_eff_mo[:nmo_ina,:] = contract('λi,λδ,δq->iq', C_ina, 2*F_ina_ao + 2*F_act_ao, C) # Fiq
            F_eff_mo[nmo_ina:nmo_ina+nmo_act,:] = contract('tu,λu,λσ,σq->tq', D1_act_mo, C_act, F_ina_ao, C) # Ftq (1)
            F_eff_mo[nmo_ina:nmo_ina+nmo_act,:] += contract('tuvw,λuvw,λq->tq', D2_act_mo, g_λuvw, C) # Ftq (2)

            # add the sr xc part as well
            n, E_xc, V_xc_sr = mf._numint.nr_rks(mol, mf.grids, mf.xc, D1_ina_ao + D1_act_ao)
            J_tot_sr = mf.get_j(dm=D1_ina_ao+D1_act_ao, omega=-omega)
            V_Hxc_sr = J_tot_sr + V_xc_sr

            F_eff_mo[:nmo_ina,:] += contract('λi,λδ,δq->iq', C_ina, V_Hxc_sr, C) # Fiq
            F_eff_mo[nmo_ina:nmo_ina+nmo_act,:] += contract('tu,λu,λσ,σq->tq', D1_act_mo, C_act, V_Hxc_sr, C) # Ftq

            # _F_eff_mo = fock_eff(h_mo, g_mo, D1_mo, D2_mo)

            # form the gradient
            G = 2 * (F_eff_mo - F_eff_mo.T)
            # print('G:\n', G)

            # check convergence
            e_vec = np.reshape(G, -1)
            error = np.linalg.norm(e_vec)

        print(f'{n_iter:4d}    {SS:7.2f}     {E_corr:12.10f}    {E_history[-1]:12.10f}   {error:+10.2e}')

        converged = np.abs(error) < threshold
        if converged:
            print('Converged!')
            break

        if orb_opt:
            # Approximate Hessian from Chaban 1997
            F_tot_pp = contract("pq,pm,qm->m", 2*F_ina_ao + 2*F_act_ao, C, C) # Diagonal of 2*Fin+2*Fact in MO basis
            F_eff_pp = np.diagonal(F_eff_mo) # Diagonal of the effective Fock matrix

            H_app = np.zeros_like(F_act_ao)

            # Hia = 2*Ftot_aa - 2*Ftot_ii
            H_app[:nmo_ina, nmo_ina+nmo_act:] = 2*F_tot_pp[nmo_ina+nmo_act:] - 2*F_tot_pp[:nmo_ina].reshape(-1,1)
            # Hta = Dact_tt*Fin_aa - 2*Feff_tt + Dact_tt*Fact_aa
            #     = Dact_tt*(Fin_aa + Fact_aa) - 2*Feff_tt
            #     = Dact_tt*(Ftot_aa) - 2*Feff_tt
            H_app[nmo_ina:nmo_ina+nmo_act, nmo_ina+nmo_act:] += contract('tt,a->ta', D1_act_mo, F_tot_pp[nmo_ina+nmo_act:])
            H_app[nmo_ina:nmo_ina+nmo_act, nmo_ina+nmo_act:] += -2*F_eff_pp[nmo_ina:nmo_ina+nmo_act].reshape(-1, 1)
            # H_app[nmo_ina:nmo_ina+nmo_act, :] += -2*F_eff_pp[nmo_ina:nmo_ina+nmo_act].reshape(-1, 1)
            # Hit = 2*Ftot_tt - 2*Ftot_ii + Dact_tt*Fin_ii - 2*Ftot_tt
            H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] += 2*F_tot_pp[nmo_ina:nmo_ina+nmo_act]
            H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] -= 2*F_tot_pp[:nmo_ina].reshape(-1,1)
            H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] += contract('tt,i->it', D1_act_mo, F_tot_pp[:nmo_ina]) #.reshape(-1,1)
            H_app[:nmo_ina, nmo_ina:nmo_ina+nmo_act] -= 2*F_eff_pp[nmo_ina:nmo_ina+nmo_act]
            H_app += H_app.T
            H_app[:nmo_ina,:nmo_ina] = 1 # To avoid division by 0
            H_app[nmo_ina:nmo_ina+nmo_act,nmo_ina:nmo_ina+nmo_act] = 1 # To avoid division by 0
            H_app[nmo_ina+nmo_act:,nmo_ina+nmo_act:] = 1 # To avoid division by 0

            # print(H_app)

            X = G / H_app
            expX = sp.linalg.expm(X)
            C = np.matmul(C, expX)


    # out of the iterative loop
    if not converged:
        print('Not converged!')

    # return in any case the last density matrix and energy
    return E_history[-1], D1_act_mo_history[-1], C


In [32]:
# _ , D, C = casscf(mf_dft, 2, 2, [8,9], max_iter=50, threshold=1e-6, alpha=None, fci_solver=None, debug=False)
_, D, C = casscf(mf_dft, 6, 6, [5,6,7,8,9,10])

Inactive orbitals: [1 2 3 4]
Active orbitals: [ 5  6  7  8  9 10]
Inactive occupations:
 [2. 2. 2. 2.]
Active occupations:
 [2. 2. 2. 0. 0. 0.]

Iter    <Ψ|S^2|Ψ>    Corr. energy      Total energy          |G|
   1       0.00     -0.2111953616    -108.2084051987    -2.11e-01
   2       0.00     -0.2107652456    -108.2079750828    +4.30e-04
   3       0.00     -0.2107651215    -108.2079749586    +1.24e-07
Converged!


In [18]:
type(umf)

pyscf.scf.uhf.UHF

In [ ]:
_ , D, C = casscf(mf, 2, 2, [8,9], max_iter=50, threshold=1e-6, alpha=None, fci_solver=None, debug=False)

Inactive orbitals: [1 2 3 4 5 6 7]
Active orbitals: [8 9]
Inactive occupations:
 [2. 2. 2. 2. 2. 2. 2.]
Active occupations:
 [2. 0.]

Iter    <Ψ|S^2|Ψ>    Corr. energy      Total energy          |G|
   1       0.00     0.0000000000    -77.1082860923    +5.96e-02
   2       0.00     0.0000000000    -77.1105209033    +1.71e-02
   3       0.00     0.0000000000    -77.1108570687    +1.04e-02
   4       0.00     0.0000000000    -77.1109245244    +6.56e-03
   5       0.00     0.0000000000    -77.1109405994    +3.76e-03
   6       0.00     0.0000000000    -77.1109446961    +2.05e-03
   7       0.00     0.0000000000    -77.1109457653    +1.09e-03
   8       0.00     0.0000000000    -77.1109460470    +5.68e-04
   9       0.00     0.0000000000    -77.1109461215    +2.96e-04
  10       0.00     0.0000000000    -77.1109461412    +1.53e-04
  11       0.00     0.0000000000    -77.1109461464    +7.94e-05
  12       0.00     0.0000000000    -77.1109461478    +4.10e-05
  13       0.00     0.0000000000 

In [7]:
mc = mcscf.CASSCF(mf, 2, 2)
mc.kernel()

CASSCF energy = -77.1109461481102
CASCI E = -77.1109461481102  E(CI) = -1.20237697906430  S^2 = 0.0000000


(np.float64(-77.11094614811017),
 np.float64(-1.2023769790642973),
 FCIvector([[ 0.9684294, -0.       ],
            [-0.       , -0.249288 ]]),
 array([[ 0.70182047,  0.70142136, -0.17836724, -0.13637508,  0.        , -0.01490273,  0.        , -0.        , -0.        , -0.        , -0.16898501, -0.12408725,  0.        ,  0.11245211],
        [ 0.0200345 ,  0.03128684,  0.46919534,  0.41388187, -0.        ,  0.02220508, -0.        ,  0.        ,  0.        ,  0.        ,  1.06240799,  0.82662714, -0.        , -0.87381435],
        [-0.        ,  0.        , -0.        ,  0.        , -0.02961519, -0.        , -0.03873971,  0.6319837 , -0.80719669, -0.12264375,  0.        , -0.        , -0.07563288, -0.        ],
        [-0.        ,  0.        , -0.        ,  0.        ,  0.39463934,  0.        ,  0.39041397,  0.06986577, -0.07159651,  0.69801767, -0.        ,  0.        ,  0.948267  , -0.        ],
        [ 0.00213747, -0.00443316, -0.11587445,  0.19910932,  0.        , -0.5005909 , 

array([[1.87571219, 0.        ],
       [0.        , 0.12428781]])